## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Bitrix

### Links: 
___

In [ ]:
(
    ID UInt64,
    OWNER_ID UInt64,
    OWNER_TYPE String,
    PRODUCT_ID UInt64,
    PRODUCT_NAME String,
    ORIGINAL_PRODUCT_NAME String,
    PRICE Float64,
    PRICE_EXCLUSIVE Float64,
    PRICE_NETTO Float64,
    PRICE_BRUTTO Float64,
    PRICE_ACCOUNT Float64,
    QUANTITY Float64,
    DISCOUNT_TYPE_ID Int32,
    DISCOUNT_RATE Float64,
    DISCOUNT_SUM Float64,
    TAX_RATE Float64,
    TAX_INCLUDED UInt8,        -- булево (0/1)
    CUSTOMIZED UInt8,          -- булево (0/1)
    MEASURE_CODE String,
    MEASURE_NAME String,
    SORT Int32,
    XML_ID String,
    TYPE String,
    STORE_ID UInt64,
    RESERVE_ID UInt64,
    DATE_RESERVE_END DateTime, -- дата/время окончания резерва
    RESERVE_QUANTITY Float64,
    DEAL_ID UInt64
)

In [57]:
# creating a table from s3
query_text = """--sql 
CREATE TABLE db1.productrows_dir_bitrix
(
    ID UInt64,
    OWNER_ID UInt64,
    OWNER_TYPE String,
    PRODUCT_ID UInt64,
    PRODUCT_NAME String,
    ORIGINAL_PRODUCT_NAME String,
    PRICE Float64,
    PRICE_EXCLUSIVE Float64,
    PRICE_NETTO Float64,
    PRICE_BRUTTO Float64,
    PRICE_ACCOUNT Float64,
    QUANTITY Float64,
    DISCOUNT_TYPE_ID Int32,
    DISCOUNT_RATE Float64,
    DISCOUNT_SUM Float64,
    TAX_RATE String,
    TAX_INCLUDED String,        -- булево (0/1)
    CUSTOMIZED String,          -- булево (0/1)
    MEASURE_CODE String,
    MEASURE_NAME String,
    SORT String,
    XML_ID String,
    TYPE String,
    STORE_ID String,
    RESERVE_ID String,
    DATE_RESERVE_END String, -- дата/время окончания резерва
    RESERVE_QUANTITY String,
    DEAL_ID String
)
ENGINE = S3('https://storage.yandexcloud.net/aggregated-data/bitrix_productrows/*.csv','CSVWithNames')
SETTINGS format_csv_delimiter = ';'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [58]:
query_text = """--sql
    CREATE TABLE db1.productrows_dir_bitrix_ch
    (
        ID UInt64,
        OWNER_ID UInt64,
        OWNER_TYPE String,
        PRODUCT_ID UInt64,
        PRODUCT_NAME String,
        ORIGINAL_PRODUCT_NAME String,
        PRICE Float64,
        PRICE_EXCLUSIVE Float64,
        PRICE_NETTO Float64,
        PRICE_BRUTTO Float64,
        PRICE_ACCOUNT Float64,
        QUANTITY Float64,
        DISCOUNT_TYPE_ID Int32,
        DISCOUNT_RATE Float64,
        DISCOUNT_SUM Float64,
        TAX_RATE String,
        TAX_INCLUDED String,        -- булево (0/1)
        CUSTOMIZED String,          -- булево (0/1)
        MEASURE_CODE String,
        MEASURE_NAME String,
        SORT String,
        XML_ID String,
        TYPE String,
        STORE_ID String,
        RESERVE_ID String,
        DATE_RESERVE_END String, -- дата/время окончания резерва
        RESERVE_QUANTITY String,
        DEAL_ID String
)
    ENGINE = MergeTree()
    ORDER BY ID
    
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [59]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.productrows_dir_bitrix_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.productrows_dir_bitrix_ch AS
    SELECT
        *
    FROM db1.productrows_dir_bitrix
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [60]:
query_text = """--sql
    SELECT
        *
    FROM  db1.productrows_dir_bitrix_mv
    limit 100
    """

ch.query_run(query_text)


ID,OWNER_ID,OWNER_TYPE,PRODUCT_ID,PRODUCT_NAME,ORIGINAL_PRODUCT_NAME,PRICE,PRICE_EXCLUSIVE,PRICE_NETTO,PRICE_BRUTTO,PRICE_ACCOUNT,QUANTITY,DISCOUNT_TYPE_ID,DISCOUNT_RATE,DISCOUNT_SUM,TAX_RATE,TAX_INCLUDED,CUSTOMIZED,MEASURE_CODE,MEASURE_NAME,SORT,XML_ID,TYPE,STORE_ID,RESERVE_ID,DATE_RESERVE_END,RESERVE_QUANTITY,DEAL_ID
i64,i64,str,i64,str,str,f64,f64,f64,f64,f64,f64,i64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str
415,2327,"""D""",1,"""Умный домофон «Спутник» версия…","""Умный домофон «Спутник» версия…",8800.0,8800.0,8800.0,8800.0,8800.0,1.0,2,0.0,0.0,"""0.0""","""Y""","""Y""","""796""","""Штука""","""10""","""""","""1""","""""","""""","""""","""""","""2327"""
593,2307,"""D""",1,"""Умный домофон «Спутник» версия…","""Умный домофон «Спутник» версия…",7900.0,7900.0,8800.0,8800.0,7900.0,1.0,2,10.23,900.0,"""0.0""","""Y""","""Y""","""796""","""Штука""","""10""","""""","""1""","""""","""""","""""","""""","""2307"""
597,2321,"""D""",1,"""Умный домофон «Спутник» версия…","""Умный домофон «Спутник» версия…",8800.0,8800.0,8800.0,8800.0,8800.0,4.0,2,0.0,0.0,"""0.0""","""Y""","""Y""","""796""","""Штука""","""10""","""""","""1""","""""","""""","""""","""""","""2321"""
629,2317,"""D""",1,"""Умный домофон «Спутник» версия…","""Умный домофон «Спутник» версия…",8800.0,8800.0,8800.0,8800.0,8800.0,12.0,2,0.0,0.0,"""0.0""","""Y""","""Y""","""796""","""Штука""","""10""","""""","""1""","""""","""""","""""","""""","""2317"""
679,2335,"""D""",1,"""Умный домофон «Спутник» версия…","""Умный домофон «Спутник» версия…",7900.0,7900.0,8800.0,8800.0,7900.0,1.0,2,10.23,900.0,"""0.0""","""Y""","""Y""","""796""","""Штука""","""10""","""""","""1""","""""","""""","""""","""""","""2335"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3355,2617,"""D""",1,"""Умный домофон «Спутник» версия…","""Умный домофон «Спутник» версия…",7899.76,7899.76,8800.0,8800.0,7899.76,1.0,2,10.23,900.24,"""0.0""","""Y""","""Y""","""796""","""Штука""","""0""","""""","""1""","""""","""""","""""","""""","""2617"""
3357,2617,"""D""",2157,"""Блок питания Nikola""","""Блок питания Nikola""",860.0,860.0,860.0,860.0,860.0,1.0,2,0.0,0.0,"""0.0""","""Y""","""Y""","""796""","""Штука""","""0""","""""","""1""","""""","""""","""""","""""","""2617"""
3553,2421,"""D""",315,"""Корпус""","""Корпус""",1000.0,1000.0,1000.0,1000.0,1000.0,2.0,2,0.0,0.0,"""0.0""","""Y""","""Y""","""796""","""Штука""","""10""","""""","""1""","""""","""""","""""","""""","""2421"""


### drop mv

In [54]:
query_text = """--sql
    DROP TABLE db1.productrows_dir_bitrix_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.productrows_dir_bitrix_ch
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.cameras_dir_asgard_mv
"""

ch.query_run(query_text)